# Perceptron
This notebook contains the implementation for a practical exercise on perceptron studies found on an [Artificial Neural Network book](https://www.amazon.com.br/Neurais-Artificiais-Engenharia-Ci%C3%AAncias-Aplicadas/dp/8588098539), section 3.6. 

The training set contains information o 3 features extracted from a oil destilation process and 1 target value indicating whether registers belong to one of 2 classes {P1 and P2}, denoted by [-1, 1] respectively. The test set contains only the features of another set of data.



## Imports

In [2]:
import random 
import pandas as pd
import numpy as np

## Loading Dataset

In [231]:
df_train = pd.read_csv('./Datasets/ex3_6_train.tsv', sep='\t')
df_train.drop(['sample'], axis=1, inplace=True)
df_test = pd.read_csv('./Datasets/ex3_6_test.tsv', sep='\t')
df_test.drop(['sample'], axis=1, inplace=True)
df_train.head()

,x1,x2,x3,target
0,-0.65,0.11,4.00,-1.0
1,-1.45,0.89,4.40,-1.0
2,2.09,0.69,12.07,-1.0
3,0.26,1.15,7.80,1.0
4,0.64,1.02,7.04,1.0


In [232]:
df_test.head()

,x1,x2,x3
0,-0.37,0.06,5.99
1,-0.78,1.13,5.59
2,0.30,0.56,5.82
3,0.78,1.06,8.07
4,0.16,0.80,6.30


In [233]:
x_train, y_train = df_train.drop(['target'], axis=1).values, df_train['target'].values
x_test = df_test.values

## Creating Perceptron Class

In [234]:
class Perceptron:
    def __init__(self, activation = 'tanh', learning_rate = 0.01, seed = None, beta = None): 
        self.activation = activation
        self.learning_rate = learning_rate
        self.seed = seed
        self.x = None
        self.x_pred = None
        self.w = None        
        self.beta = beta
        self.g = self.get_activation(self.activation, self.beta)
        
    def get_activation(self, activation, beta = None):
        """Returns an activation function
            :param activation (str): the name of the function 
                ['linear', 'unipolar_step', 'bipolar_step', 
                'logistic', 'simmetric_ramp', 'tanh', 'relu']
            :return (lambda function): the implemented activation function
        """
        if activation == 'linear':
            g = lambda x: x
        elif activation == 'unipolar_step':
            g = lambda x: 1 if x >= 0 else 0
        elif activation == 'bipolar_step':
            g = lambda x: 1 if x >= 0 else -1
        elif activation == 'logistic':
            g = lambda x, beta: 1/(1 + np.exp(-beta*x))
        elif activation == 'simmetric_ramp':
            g = lambda x, beta: x if x > -beta or x < beta else beta
        elif activation == 'tanh':
            g = lambda x, beta: (1 - np.exp(-beta*x))/(1 + np.exp(-beta*x))
        elif activation == 'relu':
            g = lambda x: x if x > 0 else 0
        else:
            raise NotImplemented
        return g
        
    def train(self, features, target, max_epochs = 30):
        """ Trains a single neuron perceptron model.
            :param features (np.array): an array containg training examples and its features
            :param target (np.array): the true values of the output 
            :max_epochs (int): the maximum number of epochs to train the algorithm
        """
        # Appending a bias constant to the features array
        self.x = np.array([np.concatenate(([1], i)) for i in features])
        
        # Initializing the weights with a random uniform function (0, 1)
        self.w = np.array([random.uniform(0,1) for i in np.arange(self.x.shape[1]-1)])
        self.w = np.concatenate(([-1], self.w))
        
        epoch = 1
        print ("Epoch {} >> W = {}".format(epoch, self.w))
        
        # Starting training until max_epochs is reached or no error is found
        keep_training = True
        while (keep_training):          
            keep_training = False            
            for index, sample in enumerate(self.x):
                u = np.dot(sample, p.w)
                y = self.g(u)                
                if (y != target[index]):
                    self.w = self.w + self.learning_rate*(target[index]-y)*sample
                    keep_training = True
            epoch += 1            
            if epoch > max_epochs:
                keep_training = False
        print ("Epoch {} >> W = {}".format(epoch, self.w))
        
    def predict(self, features):
        """ Predicts the output of a set of test features from a pre-trained model
            :params features (np.array): the test set of features
            :return y_pred (np.array): the predicted output
        """
        self.x_pred = np.array([np.concatenate(([1], i)) for i in features])
        u = np.dot(self.x_pred,p.w)
        self.y_pred = list()
        for u_i in u:
            self.y_pred.append(self.g(u_i))
        self.y_pred = np.array(self.y_pred)
        return self.y_pred
        

In [230]:
for training_index in np.arange(1, 6, 1):
    print ("\n### Starting training ", training_index)
    p = Perceptron(activation='bipolar_step', learning_rate=0.01)        
    p.train(features=x_train, target=y_train, max_epochs=2000)      
    print ("Predictions: ", p.predict(x_test))



### Starting training  1
Epoch 1 >> W = [-1.          0.98216311  0.10435385  0.34794601]
Epoch 459 >> W = [ 3.1         1.58116311  2.53535385 -0.72665399]
Predictions:  [-1  1  1  1  1  1 -1  1 -1 -1]

### Starting training  2
Epoch 1 >> W = [-1.          0.59635098  0.63210285  0.25392344]
Epoch 426 >> W = [ 3.04        1.51815098  2.49710285 -0.70047656]
Predictions:  [-1  1  1  1  1  1 -1  1 -1 -1]

### Starting training  3
Epoch 1 >> W = [-1.          0.04644378  0.90591823  0.46864117]
Epoch 426 >> W = [ 3.          1.53424378  2.45631823 -0.70235883]
Predictions:  [-1  1  1  1  1  1 -1  1 -1 -1]

### Starting training  4
Epoch 1 >> W = [-1.          0.48076392  0.08691916  0.3403141 ]
Epoch 420 >> W = [ 3.          1.54636392  2.45971916 -0.7022859 ]
Predictions:  [-1  1  1  1  1  1 -1  1 -1 -1]

### Starting training  5
Epoch 1 >> W = [-1.          0.90883605  0.36047595  0.04567482]
Epoch 405 >> W = [ 2.98        1.53663605  2.45707595 -0.69852518]
Predictions:  [-1  1  1  1

After training and prediction 5 times the algorithms no the given train and test set, we can see that, for each round, the output can be different. This is due to the fact that the weights of the perceptron are initialized randomly, which also yields to a different number of training epochs as shown in the log above. 

Given that the number of epochs did not reach the maximum number of epochs stablished in the model initialization (2000), we can conclude that the perceptron did manage to separate all classes from the train set. As a result, it is possible to affirm that such classes are linearly separable

_______________